# ReversingLabs SDK Advanced Search 

This notebook demonstrates how to use the ReversingLabs SDK to search and analyze samples using the `AdvancedSearch` and `AdvancedActions` classes. 
The Advanced Search enables users to filter samples by search criteria submitted in a query string. The script focuses on specific URI and/or file type, wide range of search keywords is available, and they can be combined using search operators to build advanced queries.
Advanced Actions is a class containing advanced and combined actions utilizing various different classes such as Static analysis (TCA-0104) and Dynamic analysis (TCA-0106). 
Combined together, client can have a comprehensive enriched report, providing single URL, File type or any other supported filter value.

For a similar implementation reference, see the [ReversingLabs SDK Cookbook - Spectra Intelligence Search Notebook](https://alt-gitlab.rl.lan/integrations/sdk/reversinglabs-sdk-cookbook/-/blob/main/Spectra Intelligence/search.ipynb?ref_type=heads).

#  1. Importing the required classes
First, we will import the required API classes from the ticloud module.

In [ ]:
from ReversingLabs.SDK.helper import *
from ReversingLabs.SDK.ticloud import AdvancedSearch, AdvancedActions

#  2. Loading the credentials

- Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md)
Next, we will load our Spectra Intelligence credentials from the local ticloud_credentials.json file.
NOTE: Instead of doing this step, you can paste your credentials while creating the Python object in the following step.

In [ ]:
import json
import re

# ---------------------------------------------------
# Configuration
# ---------------------------------------------------
SERVER = "<server>"
USERNAME = "username"
PASSWORD = "password"
USER_AGENT= "ReversingLabs SDK Cookbook v2.9.0"

# 3. Filter query string
This code block defines a Python dictionary named QUERY_STRING that sets up the parameters for an API query to the ReversingLabs platform. When running this in a Jupyter Notebook, it forms the basis for the search request by specifying filters, pagination, and the desired response format.
Addiditonal options available here: https://docs.reversinglabs.com/SpectraIntelligence/API/MalwareHunting/tca-0320

In [ ]:
QUERY_STRING = 'firstseen:[2025-02-20T00:00:00Z TO *] classification:[malicious, suspicious] filetype:EXE uri:"https://api.telegram.org/bot*" size:[0 TO *]'
# Example for URI: "https://api.telegram.org/bot*"

# 4. Extract URL prefix from a string query
URL Prefix Extraction Function
- Extracts the URL prefix from the query string using regex
- Helps identify which URLs to look for in the enriched data

In [ ]:

def extract_url_prefix_from_string_query(query_string):
    uri_pattern = r'uri:"?([^\s"]*)'
    match = re.search(uri_pattern, query_string)
    if match:
        url_pattern = match.group(1)
        # Remove the wildcard asterisk if present
        return url_pattern.replace("*", "")
    return None

# 5. Recursive Search Function
- Searches deeply through nested JSON data from the enriched report
- Finds all URLs that match the prefix pattern from the query

In [ ]:
def recursive_search_for_urls(obj, prefix):
    found = []
    if isinstance(obj, dict):
        for key, value in obj.items():
            found.extend(recursive_search_for_urls(value, prefix))
    elif isinstance(obj, list):
        for item in obj:
            found.extend(recursive_search_for_urls(item, prefix))
    elif isinstance(obj, str):
        if obj.startswith(prefix):
            found.append(obj)
    return found

# 6. Main Execution Function
- Performs the search using the ReversingLabs SDK
- Processes each sample to extract basic information
- Attempts to enrich each sample and extract relevant URLs
- Handles errors gracefully, continuing even if enrichment fails for some samples

In [ ]:
def main():
    query_string = QUERY_STRING
    url_prefix = extract_url_prefix_from_string_query(query_string)
    
    search_client = AdvancedSearch(
        host=SERVER,
        username=USERNAME,
        password=PASSWORD,
        verify=True,
        proxies=None,
        user_agent="USER_AGENT",
        allow_none_return=False
    )

    try:
        results = search_client.search_aggregated(
            query_string=query_string,
            sorting_criteria="firstseen",
            sorting_order="desc",
            max_results=100,
            records_per_page=100
        )
    except Exception as e:
        print(f"Error during search: {e}")
        return

    print(f"Total samples returned: {len(results)}")
    if not results:
        print("No samples found.")
        return

    actions = AdvancedActions(
        host=SERVER,
        username=USERNAME,
        password=PASSWORD,
        verify=True,
        proxies=None,
        user_agent="USER_AGENT",
        allow_none_return=False
    )

    minimal_results = []
    enrichment_success_count = 0
    urls_found_count = 0
    
    for sample in results:
        sha1 = sample.get("sha1")
        if not sha1:
            continue

        sample_type = sample.get("sampletype") or sample.get("filetype")
        minimal_data = {
            "hashes": {
                "sha1": sample.get("sha1"),
                "sha256": sample.get("sha256", ""),
                "md5": sample.get("md5", "")
            },
            "first_seen": sample.get("firstseen"),
            "last_seen": sample.get("lastseen"),
            "sampletype": sample_type,
            "file_size": sample.get("size"),
            "classification": sample.get("classification"),
            "threatname": sample.get("threatname"),
            "extracted_urls": []
        }

        try:
            enriched_report = actions.enriched_file_analysis(sha1)
            found_urls = recursive_search_for_urls(enriched_report, url_prefix)
            if found_urls:
                minimal_data["extracted_urls"] = list(set(found_urls))  # Deduplicate
                urls_found_count += 1
            enrichment_success_count += 1
        except Exception as e:
            print(f"Error enriching sample {sha1}: {e}")
        
        minimal_results.append(minimal_data)

    print(f"Found URLs in {urls_found_count} samples")

# 7. Results Processing and output
- Groups samples by the extracted URLs
- Creates a default group if no URLs are found
- Builds the final report structure 
- Writes the grouped results to a JSON file
- Provides summary statistics on the console

In [ ]:
url_groups = {}
for sample in minimal_results:
    for url in sample.get("extracted_urls", []):
        if url not in url_groups:
            url_groups[url] = []
        url_groups[url].append(sample)

if not url_groups and minimal_results:
    print("No URLs found in any samples. Creating a default group for all samples.")
    default_url = f"{url_prefix}[no_specific_url_found]"
    url_groups[default_url] = minimal_results

grouped_output = {"urls": []}
for url, samples in url_groups.items():
    hashes = [sample["hashes"]["sha1"] for sample in samples]
    
    grouped_output["urls"].append({
        "value": url,
        "hashes": hashes,
        "samples": samplesđ
    })
output_file = "report.json"
try:
    with open(output_file, "w") as f:
        json.dump(grouped_output, f, indent=2)
    print(f"Grouped report written to {output_file}")
except Exception as e:
    print("Error exporting report:", e)

if __name__ == "__main__":
    main()

# 8. Results example

In [ ]:
{
  "urls": [
    {
      "value": "https://api.telegram.org/bot",
      "hashes": [
        "54b358febcc3d090bfec348621278910ad75af22",
        "1c5b22ed6813fb0b3d89f7e8fb0434128ab02422",
        "c921a5477d28011a22ae6cdce37ce3a9de6b0f22",
        "b2343226350cbec1a50cedc6893da5b4caf6e002",
        "8a04c82985e039a529a8d1977a1d4ee1e523cf33",
        "666a04d24a813b74dc4316e383f5c29404c39766"
      ],
      "samples": [
        {
          "hashes": {
            "sha1": "54b358febcc3d090bfec348621278910ad75af3e3",
            "sha256": "a9cb0b88237caebe6eaba215ce95ca411fddeb3dsdb3a63b4b4e08e6025850e123d",
            "md5": "57a688d8e37b32caa5f8687edaed81ff"
          },
          "first_seen": "2025-02-27T10:22:22Z",
          "last_seen": "2025-02-27T10:58:54Z",
          "sampletype": "PE/.Net Exe",
          "file_size": 62464,
          "classification": "MALICIOUS",
          "threatname": "ByteCode-MSIL.Trojan.Zilla",
          "extracted_urls": [
            "https://api.telegram.org/bot"
          ]
        },
        {
          "hashes": {
            "sha1": "1c5b22ed6813fb0b3d89f7e8fb0434128adsb024c6",
            "sha256": "a0cca8f08306f700c650621a5123b27a1b86ds11ab1d16fc2281544e238b85cf05",
            "md5": "0a603ecaaa4b5fe091cc7f5cc8144a1a"
          },
          "first_seen": "2025-02-27T09:40:47Z",
          "last_seen": "2025-02-27T10:50:41Z",
          "sampletype": "PE/.Net Exe",
          "file_size": 273408,
          "classification": "MALICIOUS",
          "threatname": "Win32.Exploit.SnakeKeylogger",
          "extracted_urls": [
            "https://api.telegram.org/bot/sendMessage?chat_id=&text=",
            "https://api.telegram.org/bot/sendMessage?chat_id=&text=%20%0D%0A%0D%0APC%20Name:648351%0D%0ADate%20and%20Time:%202/28/2025%20/%201:38:33%20PM%0D%0ACountry%20Name:%20%0D%0A",
            "https://api.telegram.org/bot"
          ]
        },
        {
          "hashes": {
            "sha1": "c921a5477d28011a22ae6cdce37ceds3a9de6b0f66",
            "sha256": "e3a31d84fc68d5b66fa85fe824437c144e6a130008a6bbe49d3cdsd24fd854d679b",
            "md5": "b2a9e38cf99f9a71361a11624467d884"
          },
          "first_seen": "2025-02-27T06:58:00Z",
          "last_seen": "2025-02-27T09:51:56Z",
          "sampletype": "PE/.Net Exe",
          "file_size": 75264,
          "classification": "MALICIOUS",
          "threatname": "Win32.Exploit.Xworm",
          "extracted_urls": [
            "https://api.telegram.org/bot",
            "https://api.telegram.org/bot7921537821"
          ]
        },
        {
          "hashes": {
            "sha1": "b2343226350cbec1a50cedc6893dasd5b4caf6e004",
            "sha256": "51e9fba0982af1107327f0aa5f301c6e81sd38c93369fb442aae2d67f631de0410",
            "md5": "38a2071959f039f1cfdc7f0c59d4e5d1"
          },
          "first_seen": "2025-02-27T06:38:49Z",
          "last_seen": "2025-02-27T07:30:57Z",
          "sampletype": "PE/.Net Exe",
          "file_size": 81408,
          "classification": "MALICIOUS",
          "threatname": "Win32.Exploit.Xworm",
          "extracted_urls": [
            "https://api.telegram.org/bot6200639583",
            "https://api.telegram.org/bot"
          ]
        },
        {
          "hashes": {
            "sha1": "8a04c82985e039a529a8d1977a1d4eesd1e523cfbe",
            "sha256": "349f57d068cb3b4685781cf46a608319e92e52csd48ebb52112ea2b6cca9f43995",
            "md5": "3faff391fb6b4ce94cea5dabf5590e70"
          },
          "first_seen": "2025-02-26T22:25:32Z",
          "last_seen": "2025-02-26T22:58:05Z",
          "sampletype": "PE/.Net Exe",
          "file_size": 203776,
          "classification": "MALICIOUS",
          "threatname": "Win32.Exploit.XWorm",
          "extracted_urls": [
            "https://api.telegram.org/bot"
          ]
        },
        {
          "hashes": {
            "sha1": "666a04d24a813b74dc4316e383f5c29404csd39784",
            "sha256": "7dfcfd0a2c51d21ca3e3cd18241a02ca0b855sdsdae220ffb3b39791dadf89541e22",
            "md5": "e72c66128288dd176266f7cd75d691c9"
          },
          "first_seen": "2025-02-26T18:47:38Z",
          "last_seen": "2025-02-26T18:53:20Z",
          "sampletype": "PE/.Net Exe",
          "file_size": 38400,
          "classification": "MALICIOUS",
          "threatname": "ByteCode-MSIL.Trojan.Zilla",
          "extracted_urls": [
            "https://api.telegram.org/bot"
          ]
        }
      ]
    },
    {
      "value": "https://api.telegram.org/bot/sendMessage?chat_id=&text=",
      "hashes": [
        "1c5b22ed6813fb0b3d89f7e8fb0434128ab024c6"
      ],
      "samples": [
        {
          "hashes": {
            "sha1": "1c5b22ed6813fb0b3d89f7e8fb0434128ab02sd4c6",
            "sha256": "a0cca8f08306f700c650621a5123b27a1b8sd611ab1d16fc2281544e238b85cf05",
            "md5": "0a603ecaaa4b5fe091cc7f5cc8144a1a"
          },
          "first_seen": "2025-02-27T09:40:47Z",
          "last_seen": "2025-02-27T10:50:41Z",
          "sampletype": "PE/.Net Exe",
          "file_size": 273408,
          "classification": "MALICIOUS",
          "threatname": "Win32.Exploit.SnakeKeylogger",
          "extracted_urls": [
            "https://api.telegram.org/bot/sendMessage?chat_id=&text=",
            "https://api.telegram.org/bot/sendMessage?chat_id=&text=%20%0D%0A%0D%0APC%20Name:648351%0D%0ADate%20and%20Time:%202/28/2025%20/%201:38:33%20PM%0D%0ACountry%20Name:%20%0D%0A",
            "https://api.telegram.org/bot"
          ]
        }
      ]
    },
    {
      "value": "https://api.telegram.org/bot/sendMessage?chat_id=&text=%20%0D%0A%0D%0APC%20Name:648351%0D%0ADate%20and%20Time:%202/28/2025%20/%201:38:33%20PM%0D%0ACountry%20Name:%20%0D%0A",
      "hashes": [
        "1c5b22ed6813fb0b3d89f7e8fb0434128ab024c6"
      ],
      "samples": [
        {
          "hashes": {
            "sha1": "1c5b22ed6813fb0b3d89f7e8fb0434128abfd024c6",
            "sha256": "a0cca8f08306f700c650621a5123b27a1b8fd611ab1d16fc2281544e238b85cf05",
            "md5": "0a603ecaaa4b5fe091cc7f5cc8144a1a"
          },
          "first_seen": "2025-02-27T09:40:47Z",
          "last_seen": "2025-02-27T10:50:41Z",
          "sampletype": "PE/.Net Exe",
          "file_size": 273408,
          "classification": "MALICIOUS",
          "threatname": "Win32.Exploit.SnakeKeylogger",
          "extracted_urls": [
            "https://api.telegram.org/bot/sendMessage?chat_id=&text=",
            "https://api.telegram.org/bot/sendMessage?chat_id=&text=%20%0D%0A%0D%0APC%20Name:648351%0D%0ADate%20and%20Time:%202/28/2025%20/%201:38:33%20PM%0D%0ACountry%20Name:%20%0D%0A",
            "https://api.telegram.org/bot"
          ]
        }
      ]
    },
    {
      "value": "https://api.telegram.org/bot7921537821",
      "hashes": [
        "c921a5477d28011a22ae6cdce37ce3a9de6b0f66"
      ],
      "samples": [
        {
          "hashes": {
            "sha1": "c921a5477d28011a22ae6cdce37ce3dfa9de6b0f66",
            "sha256": "e3a31d84fc68d5b66fa85fe824437cdf144e6a130008a6bbe49d3c24fd854d679b",
            "md5": "b2a9e38cf99f9a71361a11624467d884"
          },
          "first_seen": "2025-02-27T06:58:00Z",
          "last_seen": "2025-02-27T09:51:56Z",
          "sampletype": "PE/.Net Exe",
          "file_size": 75264,
          "classification": "MALICIOUS",
          "threatname": "Win32.Exploit.Xworm",
          "extracted_urls": [
            "https://api.telegram.org/bot",
            "https://api.telegram.org/bot7921537821"
          ]
        }
      ]
    },
    {
      "value": "https://api.telegram.org/bot6200639583",
      "hashes": [
        "b2343226350cbec1a50cedc6893da5b4caf6e004"
      ],
      "samples": [
        {
          "hashes": {
            "sha1": "b2343226350cbec1a50cedc6893da5b4cdfaf6e004",
            "sha256": "51e9fba0982af1107327f0aa5f301c6e8dfd138c93369fb442aae2d67f631de0410",
            "md5": "38a2071959f039f1cfdc7f0c59d4e5d1"
          },
          "first_seen": "2025-02-27T06:38:49Z",
          "last_seen": "2025-02-27T07:30:57Z",
          "sampletype": "PE/.Net Exe",
          "file_size": 81408,
          "classification": "MALICIOUS",
          "threatname": "Win32.Exploit.Xworm",
          "extracted_urls": [
            "https://api.telegram.org/bot6200639583",
            "https://api.telegram.org/bot"
          ]
        }
      ]
    }
  ]
}